In [1]:
import json
import torch
import numpy as np
import faiss
from tqdm import tqdm
from transformers import BertTokenizer, BertModel

# **本地加载 BERT**
LOCAL_MODEL_PATH = "/root/autodl-tmp/all-MiniLM-L6-v2"  # 请修改为本地模型路径

# 加载本地 tokenizer 和模型
tokenizer = BertTokenizer.from_pretrained(LOCAL_MODEL_PATH)
model = BertModel.from_pretrained(LOCAL_MODEL_PATH)

# 加载文本数据
def load_text_data(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    texts = []
    for item in data:
        # 将 Disease、Symptoms 和 Treatments 组合成一个文本块
        disease = item.get("Disease", "")
        symptoms = " ".join(item.get("Symptoms", []))
        treatments = " ".join(item.get("Treatments", []))
        combined_text = f"Disease: {disease}\nSymptoms: {symptoms}\nTreatments: {treatments}"
        texts.append(combined_text)
    return texts

# 计算 BERT 文本嵌入，支持本地运行
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # 取 [CLS] 作为句子向量

# 构建 FAISS 索引，带进度条
def build_faiss_index(texts):
    print("\n计算文本嵌入中...")
    embeddings = np.array([get_embedding(text) for text in tqdm(texts, desc="嵌入计算")], dtype="float32")
    
    print("\n构建 FAISS 索引中...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index, texts

# 保存 FAISS 索引和文本数据
def save_retrieval_system(index, texts, index_file, texts_file):
    print("\n保存 FAISS 索引和文本数据...")
    faiss.write_index(index, index_file)
    
    # 将文本数据保存到新的 JSON 文件
    with open(texts_file, "w", encoding="utf-8") as f:
        json.dump(texts, f, ensure_ascii=False, indent=4)
    print("✅ 本地文本检索系统构建完成！")

# **主函数**：本地运行
def build_and_save_text_retrieval_system(json_file, index_file, texts_file):
    texts = load_text_data(json_file)
    index, texts = build_faiss_index(texts)
    save_retrieval_system(index, texts, index_file, texts_file)

# 本地数据路径（请修改为本地路径）
text_data_file = "/root/autodl-tmp/NHS_Data.json"  # 输入文件路径
text_index_file = "/root/autodl-tmp/NHS_text_index.faiss"  # 输出 FAISS 索引文件路径
text_texts_file = "/root/autodl-tmp/NHS_text_texts.json"   # 输出文本数据文件路径

# 运行：构建索引并保存
build_and_save_text_retrieval_system(text_data_file, text_index_file, text_texts_file)


计算文本嵌入中...


嵌入计算: 100%|██████████| 411/411 [00:06<00:00, 65.53it/s]


构建 FAISS 索引中...

保存 FAISS 索引和文本数据...
✅ 本地文本检索系统构建完成！
